In [3]:
# -*- coding: utf-8 -*-

import os
import json
import nltk
import gensim
import numpy as np
from gensim import corpora, models, similarities
import pickle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.recurrent import LSTM,SimpleRNN
from keras import optimizers

C:\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
#model=gensim.models.Word2Vec.load("chat_model1.w2v")
model =  gensim.models.KeyedVectors.load("./model/chat_model1-300.w2v")

In [10]:
file=open('conv.json');
data = json.load(file)
cor=data["conversations"];

x=[]
y=[]

path2="corpus";

for i in range(len(cor)):
    for j in range(len(cor[i])):
        if j<len(cor[i])-1:
            x.append(cor[i][j]);
            y.append(cor[i][j+1]);

tok_x=[]
tok_y=[]
for i in range(len(x)):
    tok_x.append(nltk.word_tokenize(x[i].lower()))
    tok_y.append(nltk.word_tokenize(y[i].lower()))
    
    
sentend=np.ones((300,),dtype=np.float32) 

In [11]:
vec_x=[]
for sent in tok_x:
    sentvec = [model[w] for w in sent if w in model.wv.vocab]
    vec_x.append(sentvec)
    
vec_y=[]
for sent in tok_y:
    sentvec = [model[w] for w in sent if w in model.wv.vocab]
    vec_y.append(sentvec)           
    
    
for tok_sent in vec_x:
    tok_sent[14:]=[]
    tok_sent.append(sentend)
    

for tok_sent in vec_x:
    if len(tok_sent)<15:
        for i in range(15-len(tok_sent)):
            tok_sent.append(sentend)    
            
for tok_sent in vec_y:
    tok_sent[14:]=[]
    tok_sent.append(sentend)
    

for tok_sent in vec_y:
    if len(tok_sent)<15:
        for i in range(15-len(tok_sent)):
            tok_sent.append(sentend)             


C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [12]:
tok_y

[['i', 'am', 'doing', 'well', ',', 'how', 'about', 'you', '?'],
 ['i', "'m", 'also', 'good', '.'],
 ['that', "'s", 'good', 'to', 'hear', '.'],
 ['yes', 'it', 'is', '.'],
 ['hi'],
 ['how', 'are', 'you', 'doing', '?'],
 ['i', 'am', 'doing', 'well', '.'],
 ['that', 'is', 'good', 'to', 'hear'],
 ['yes', 'it', 'is', '.'],
 ['can', 'i', 'help', 'you', 'with', 'anything', '?'],
 ['yes', ',', 'i', 'have', 'a', 'question', '.'],
 ['what', 'is', 'your', 'question', '?'],
 ['could', 'i', 'borrow', 'a', 'cup', 'of', 'sugar', '?'],
 ['i', "'m", 'sorry', ',', 'but', 'i', 'do', "n't", 'have', 'any', '.'],
 ['thank', 'you', 'anyway'],
 ['no', 'problem'],
 ['i', 'am', 'doing', 'well', ',', 'how', 'about', 'you', '?'],
 ['i', 'am', 'also', 'good', '.'],
 ['that', "'s", 'good', '.'],
 ['what', 'good', 'news', '?'],
 ['i', 'ca', "n't", 'read', '.'],
 ['so', 'what', "'s", 'your', 'favorite', 'color', '?'],
 ['blue'],
 ['who',
  '?',
  'who',
  'is',
  'but',
  'a',
  'form',
  'following',
  'the',
  'func

In [13]:
vec_x=np.array(vec_x)
vec_y=np.array(vec_y)

In [14]:
vec_x[0].shape

(15, 300)

In [15]:
vec_x
#Jumlah kalimat, panjang array, fitur vector
#(86,15,300)

array([[[ 0.21034324,  0.24194022,  0.29061761, ..., -0.2315835 ,
          0.18495096, -0.01219473],
        [ 0.01173872,  0.13560122, -0.3364883 , ..., -0.51618856,
         -0.15339333, -0.5202885 ],
        [ 0.08117685,  0.15714699, -0.14018328, ...,  0.02985755,
          0.01824623, -0.3056154 ],
        ...,
        [ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ],
        [ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ],
        [ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ]],

       [[-0.01304407,  0.14175303, -0.3062282 , ..., -0.6480493 ,
          0.47340444, -0.44385377],
        [-0.17642888,  0.16060558,  0.56060416, ..., -0.5988163 ,
          0.35530677, -1.1416143 ],
        [ 0.13194172,  0.16962156,  0.07887359, ..., -0.11506379,
          0.02716614, -0.29173043],
        ...,
        [ 1.        ,  1.        ,  1.        , ...,  

In [16]:
x_train,x_test, y_train,y_test = train_test_split(vec_x, vec_y, test_size=0.2, random_state=1)
    
model=Sequential()
model.add(LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.add(LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.add(LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.add(LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss='cosine_proximity', optimizer=adam, metrics=['accuracy'])

# Train here

In [17]:
model.fit(x_train, y_train, epochs=19,validation_data=(x_test, y_test))
model.save('LSTM500.h5');

Train on 68 samples, validate on 18 samples
Epoch 1/19
68/68 [==============================] - 3s 40ms/step - loss: -0.6560 - acc: 0.0000e+00 - val_loss: -0.6293 - val_acc: 0.0000e+00
Epoch 2/19
68/68 [==============================] - 0s 6ms/step - loss: -0.6610 - acc: 0.0000e+00 - val_loss: -0.6341 - val_acc: 0.0148
Epoch 3/19
68/68 [==============================] - 0s 6ms/step - loss: -0.6653 - acc: 0.0078 - val_loss: -0.6384 - val_acc: 0.0185
Epoch 4/19
68/68 [==============================] - 0s 6ms/step - loss: -0.6689 - acc: 0.0216 - val_loss: -0.6420 - val_acc: 0.0222
Epoch 5/19
68/68 [==============================] - 0s 6ms/step - loss: -0.6717 - acc: 0.0216 - val_loss: -0.6450 - val_acc: 0.0222
Epoch 6/19
68/68 [==============================] - 0s 7ms/step - loss: -0.6740 - acc: 0.0216 - val_loss: -0.6475 - val_acc: 0.0222
Epoch 7/19
68/68 [==============================] - 0s 7ms/step - loss: -0.6759 - acc: 0.0216 - val_loss: -0.6495 - val_acc: 0.0222
Epoch 8/19
68/68 [=

# Try Predict

In [20]:
predictions=model.predict(x_test) 
mod = gensim.models.Word2Vec.load('./model/chat_model1-300.w2v');   
[mod.most_similar([predictions[13][i]])[0] for i in range(15)]

C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\envs\tensorflow\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('sepertinya', 0.2254774272441864),
 ('Umm', 0.2507757544517517),
 ('Umm', 0.25366780161857605),
 ('Umm', 0.2556420862674713),
 ('Umm', 0.25656020641326904),
 ('Umm', 0.2569032311439514),
 ('Umm', 0.25693371891975403),
 ('Umm', 0.2568012773990631),
 ('Umm', 0.2565876841545105),
 ('Umm', 0.2563498020172119),
 ('Umm', 0.2560974657535553),
 ('Umm', 0.2558760643005371),
 ('Umm', 0.2556093633174896),
 ('Umm', 0.25540539622306824),
 ('Umm', 0.2552252411842346)]

In [19]:
x_test

array([[[ 0.27268437,  0.15322874, -0.06124368, ..., -0.01242189,
          0.1126594 , -0.34383303],
        [ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ],
        [ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ],
        ...,
        [ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ],
        [ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ],
        [ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ]],

       [[ 0.17942017,  0.4660895 ,  0.0480305 , ...,  0.23390172,
          0.2772649 , -0.5460838 ],
        [-0.11008591,  0.33167502,  0.03457566, ...,  0.16518685,
          0.00781686, -0.625418  ],
        [-0.3295895 ,  0.56352127,  0.4857997 , ...,  0.08393965,
         -0.20914996, -0.35437003],
        ...,
        [ 1.        ,  1.        ,  1.        , ...,  